# Reteriving data from tsetmc.com
## Total data
It is possible to retrieve latest data by sending a POST request to [this url](http://www.tsetmc.com/tsev2/data/MarketWatchPlus.aspx?h=163530&r=986388925).

In [13]:
import requests

MarketWatchPlus = "http://www.tsetmc.com/tsev2/data/MarketWatchPlus.aspx"
data = requests.post(MarketWatchPlus)

The data is binary format so we need to decode it back to unicode:

In [16]:
content = data.content.decode('utf-8')
print(content)

28447,140485,153418@@408934423224097,IRO1FRVR0001,فرآور,فرآوري‌موادمعدني‌ايران‌,163528,2493,2438,2455,11,12931,31929365,2455,2544,2436,-356,205255,794,1,27,2557.00,2315.00,120000000,300;611986653700161,IRO3SBZZ0001,سبزوا,سيمان لار سبزوار,163528,730,723,725,65,825037,596485386,718,730,727,2,1,906,2,53,763.00,691.00,599425579,303;778253364357513,IRO1BMLT0001,وبملت,بانك ملت,163528,2265,2270,2256,560,7474389,16969588847,2252,2290,2265,501,4415011,2966,1,57,2378.00,2152.00,40000000000,300;831325835570803,IRO7CNAP0001,ثنام,س ساختماني ب نام آوران مهندسي,163522,1111,1105,1100,2,20000,22110000,1100,1111,1124,,1,49,4,70,999999.00,1.00,500000000,309;1010647938388655,IRO7BHPP0001,بهپاك,صنعتي بهپاك,163520,4121,4133,4197,7,29424,121621584,4120,4197,4136,749,1,355,4,42,999999.00,1.00,320000000,309;1358190916156744,IRO1TAZB0001,وآذر,سرمايه‌گذاري‌توسعه‌آذربايجان‌,163528,3271,3128,3271,7,12037,39373027,3271,3271,3116,-163,160462,953,1,70,3271.00,2961.00,100000000,300;2254054929817435,IRO1KDPS0001,غدام,خ

The represented data consist of different parts, separated by `@`.

In [20]:
parts = content.split('@')
print(len(parts))

5


In [39]:
fast_view = parts[1]
inst_price = parts[2].split(';')

`inst_price` holds the table data, so we get a known share data as row:

In [80]:
from pprint import pprint

known_row = None
for item in inst_price:
    item = item.split(',')
    if item[0] == '3823243780502959':
        known_row = item
        pprint({item.index(elem): elem for elem in item})

{0: '3823243780502959',
 1: 'IRO1TKIN0001',
 2: 'رتكو',
 3: 'كنترل\u200cخوردگي\u200cتكين\u200cكو',
 4: '163529',
 5: '8439',
 6: '8319',
 7: '8300',
 8: '148',
 9: '409128',
 10: '3403652394',
 11: '8220',
 13: '8620',
 14: '932',
 15: '58005',
 16: '1683',
 17: '1',
 18: '74',
 19: '9051.00',
 20: '8189.00',
 21: '90000000',
 22: '300'}


Each share has a type that is in field 22 of it's row. This is the type translation as far as I could extract (This value is name as `yval` in their source code):

In [84]:
types = {
    '300': 'سهام',
    '303': 'سهام',
    '309': 'فرابورس',
    '400': 'حق تقدم',
    '403': 'حق تقدم',
    '404': 'حق تقدم',
    '301': 'اوراق مشارکت',
    '306': 'اوراق مشارکت',
    '304': 'آتی',
    '305': 'صندوق سرمای‌گذاری',
    '600': 'اختیار فروش',
    '601': 'اختیار فروش',
    '308': 'بورس کالا',
    '701': 'بورس کالا'
}

Then we try to translate this from [this page](http://www.tsetmc.com/loader.aspx?ParTree=151311&i=3823243780502959).

In [87]:
def format_row(item):
    row = dict(
        id=item[1],
        symbol=item[2],
        name=item[3],
        fprice=item[5],
        last_price=dict(
        volume=item[6],
        ),
        last_deal=dict(
            volume=item[7],
        ),
        transactions = dict(
            number=item[8],
            volume=item[9],
            value=item[10]
        ),
        min_=item[11],
        max_=item[12],
        yprice=item[13],
        eps=item[14],
        basis=item[15],
        # Not sure about it
        allowed_price=dict(
            today=item[19],
            yesterday=item[20]
        ),
        number_of_shares=item[21],
        type_=types[item[22]]
    )

    return row

pprint(format_row(known_row))

{'allowed_price': {'today': '9051.00', 'yesterday': '8189.00'},
 'basis': '58005',
 'eps': '932',
 'fprice': '8439',
 'id': 'IRO1TKIN0001',
 'last_deal': {'volume': '8300'},
 'last_price': {'volume': '8319'},
 'max_': '8439',
 'min_': '8220',
 'name': 'كنترل\u200cخوردگي\u200cتكين\u200cكو',
 'number_of_shares': '90000000',
 'symbol': 'رتكو',
 'transactions': {'number': '148', 'value': '3403652394', 'volume': '409128'},
 'type_': 'سهام',
 'yprice': '8620'}


In [88]:
from pprint import pprint

table = []
for item in inst_price:
    item = item.split(',')
    row = format_row(item)
    table.append(row)
pprint(table)

[{'allowed_price': {'today': '2557.00', 'yesterday': '2315.00'},
  'basis': '205255',
  'eps': '-356',
  'fprice': '2493',
  'id': 'IRO1FRVR0001',
  'last_deal': {'volume': '2455'},
  'last_price': {'volume': '2438'},
  'max_': '2544',
  'min_': '2455',
  'name': 'فرآوري\u200cموادمعدني\u200cايران\u200c',
  'number_of_shares': '120000000',
  'symbol': 'فرآور',
  'transactions': {'number': '11', 'value': '31929365', 'volume': '12931'},
  'type_': 'سهام',
  'yprice': '2436'},
 {'allowed_price': {'today': '763.00', 'yesterday': '691.00'},
  'basis': '1',
  'eps': '2',
  'fprice': '730',
  'id': 'IRO3SBZZ0001',
  'last_deal': {'volume': '725'},
  'last_price': {'volume': '723'},
  'max_': '730',
  'min_': '718',
  'name': 'سيمان لار سبزوار',
  'number_of_shares': '599425579',
  'symbol': 'سبزوا',
  'transactions': {'number': '65', 'value': '596485386', 'volume': '825037'},
  'type_': 'سهام',
  'yprice': '727'},
 {'allowed_price': {'today': '2378.00', 'yesterday': '2152.00'},
  'basis': '441